In [1]:
%load_ext autoreload
%autoreload 2

import os
import matplotlib.pyplot as plt

# Change working directory to project root
os.chdir('../../')

import torch
import cv2
import numpy as np
from pathlib import Path
from tqdm import tqdm
import json
import imageio
import decord
from hmr4d.utils.wis3d_utils import make_wis3d, add_motion_as_lines
from hmr4d.dataset.rich.rich_utils import get_w2az_sahmr, parse_seqname_info
from hmr4d.utils.geo_transform import apply_T_on_points

In [2]:
from hmr4d.utils.smplx_utils import make_smplx
smplx_model = make_smplx("rich-smplx", gender="neutral")
smplx_output = smplx_model()
print("smplx joint (full) shape:", smplx_output.joints.shape)

smplx_joints22 = smplx_output.joints[:, :22]
smplx_verts = smplx_output.vertices  # (1, 10475, 3)

smplx joint (full) shape: torch.Size([1, 127, 3])


In [24]:
# smplx2smpl_pkl = Path("/home/shenzehong/Code/WHAM/dataset/body_models/smplx2smpl.pkl")
# import joblib
# smplx2smpl = torch.from_numpy(joblib.load(smplx2smpl_pkl)['matrix']).unsqueeze(0).float()

smplx2smpl = torch.load("hmr4d/utils/body_model/smplx2smpl_sparse.pt").to_dense()
print("smplx2smpl shape:", smplx2smpl.shape)

smpl_verts_converted = torch.matmul(smplx2smpl, smplx_verts)  # (1, 6890, 3)
print("smpl_verts_converted shape:", smpl_verts_converted.shape)

smplx2smpl shape: torch.Size([6890, 10475])
smpl_verts_converted shape: torch.Size([1, 6890, 3])


In [26]:
import pickle

# smpl_struct = "inputs/checkpoints/body_models/smpl/SMPL_NEUTRAL.pkl"
# with open(smpl_struct, "rb") as smpl_file:
#     data_struct = pickle.load(smpl_file, encoding="latin1")
# smpl_J_regressor = torch.tensor(data_struct["J_regressor"].todense())[None].float()  # (1, 24, 6890)
smpl_J_regressor = torch.load("hmr4d/utils/body_model/smpl_neutral_J_regressor.pt")
print(smpl_J_regressor.shape)
smpl_joints22 = torch.matmul(smpl_J_regressor, smpl_verts_converted)[:, :22]  # (1, 22, 3)

torch.Size([24, 6890])


In [25]:
(smpl_joints22 - smplx_joints22).max()

tensor(0.0438)

### 储存

In [11]:
torch.save(smpl_J_regressor[0], "smpl_neutral_J_regressor.pt")

In [14]:
smplx2smpl_ = smplx2smpl[0]

# 获取非零元素的位置
nonzero_indices = torch.nonzero(smplx2smpl_, as_tuple=False)

# 获取这些位置的值
nonzero_values = smplx2smpl_[nonzero_indices[:, 0], nonzero_indices[:, 1]]

# 创建稀疏张量
sparse_smplx2smpl = torch.sparse_coo_tensor(nonzero_indices.t(), nonzero_values, smplx2smpl_.size())
torch.save(sparse_smplx2smpl, "smplx2smpl_sparse.pt")

In [15]:
a = torch.load("smplx2smpl_sparse.pt").to_dense()

